<a href="https://colab.research.google.com/github/Phonchanit/bsc_dpdm24/blob/main/midterm_dpdm24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## การเตรียมข้อมูลประกอบด้วย กำจัด outlier, แก้ missing, normalization, one-hot-encoder และ อื่นๆ ที่นักศึกษาเห็นว่าเหมาะสม

# 1 เตรียมข้อมูลเพื่อทำนายราคาบ้าน (15 คะแนน)
เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก [google drive](https://drive.google.com/drive/folders/1uIcUFY1yldIq-A3wijh2n2N6az-EljIe?usp=sharing) โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

* ```'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 	'sub_district', 'zone'``` จากไฟล์ "Data House" - บ้านมือ 2 (5 คะแนน)

* ```จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school')``` จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ  7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

* ```ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score')``` จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)
2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)
3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

# **เตรียมข้อมูล**

## **Data House**

In [69]:
# prompt: connect to my drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [70]:
# prompt: read file .xlsx

import pandas as pd

# ระบุ path ของไฟล์
data_house = '/content/drive/My Drive/Dataset/Data House.xlsx'

# อ่านข้อมูลจากชีต 'บ้านมือ2'
data = pd.read_excel(data_house, sheet_name='บ้านมือ2')

In [71]:
# เลือกเฉพาะคอลัมน์ที่ต้องการ
columns_to_keep = [
    'renovate', 'number_of_rai', 'number_of_ngan', 'number_of_va',
    'number_of_story', 'number_of_bedroom', 'number_of_bathroom',
    'direction', 'furniture', 'usable_area', 'number_of_parking',
    'sell_price', 'district', 'sub_district', 'zone'
]
filtered_data = data[columns_to_keep]

# แสดงข้อมูล 5 แถวแรก
print(filtered_data.head())

  renovate  number_of_rai  number_of_ngan  number_of_va  number_of_story  \
0       no              0               1          45.0              2.0   
1       no              0               0          88.0              1.0   
2       no              1               0           8.0              1.0   
3       no              0               0          61.6              2.0   
4       no              0               0          72.3              2.0   

   number_of_bedroom  number_of_bathroom direction furniture  usable_area  \
0                2.0                   2   ไม่ระบุ     ไม่มี        244.0   
1                1.0                   1   ไม่ระบุ     ไม่มี        164.0   
2               11.0                  11   ไม่ระบุ   ไม่ระบุ        489.0   
3                3.0                   3   ไม่ระบุ   ไม่ระบุ        203.0   
4                3.0                   2   ไม่ระบุ   ไม่ระบุ        164.0   

   number_of_parking  sell_price      district sub_district  \
0                

In [72]:
#prompt: ตรวจสอบค่าว่างในแต่ละคอลัมน์ที่เลือก
missing_values = filtered_data.isnull().sum()
print("Missing Values:\n", missing_values)

Missing Values:
 renovate              211
number_of_rai           0
number_of_ngan          0
number_of_va            0
number_of_story       549
number_of_bedroom      27
number_of_bathroom      0
direction               0
furniture               0
usable_area           920
number_of_parking     215
sell_price              1
district                0
sub_district          140
zone                  638
dtype: int64


In [73]:
# เติมค่าว่างในคอลัมน์ renovate ด้วย 'unknown'
filtered_data['renovate'] = filtered_data['renovate'].fillna('unknown')

<ipython-input-73-234004b5bf77>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['renovate'] = filtered_data['renovate'].fillna('unknown')


In [74]:
# แสดงข้อมูล 5 แถวแรก
print(filtered_data.head())

  renovate  number_of_rai  number_of_ngan  number_of_va  number_of_story  \
0       no              0               1          45.0              2.0   
1       no              0               0          88.0              1.0   
2       no              1               0           8.0              1.0   
3       no              0               0          61.6              2.0   
4       no              0               0          72.3              2.0   

   number_of_bedroom  number_of_bathroom direction furniture  usable_area  \
0                2.0                   2   ไม่ระบุ     ไม่มี        244.0   
1                1.0                   1   ไม่ระบุ     ไม่มี        164.0   
2               11.0                  11   ไม่ระบุ   ไม่ระบุ        489.0   
3                3.0                   3   ไม่ระบุ   ไม่ระบุ        203.0   
4                3.0                   2   ไม่ระบุ   ไม่ระบุ        164.0   

   number_of_parking  sell_price      district sub_district  \
0                

In [75]:
filtered_data['number_of_story'].fillna(filtered_data['number_of_story'].median(), inplace=True)

<ipython-input-75-86df8f728049>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_data['number_of_story'].fillna(filtered_data['number_of_story'].median(), inplace=True)
<ipython-input-75-86df8f728049>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['number_of_story'].fillna(filtered_data['number_of_story'].median(), inp

In [76]:
filtered_data['number_of_bedroom'].fillna(filtered_data['number_of_bedroom'].median(), inplace=True)

<ipython-input-76-1515ce7701b5>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_data['number_of_bedroom'].fillna(filtered_data['number_of_bedroom'].median(), inplace=True)
<ipython-input-76-1515ce7701b5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['number_of_bedroom'].fillna(filtered_data['number_of_bedroom'].media

In [77]:
filtered_data['usable_area'].fillna(filtered_data['usable_area'].median(), inplace=True)

<ipython-input-77-31d0a5925258>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_data['usable_area'].fillna(filtered_data['usable_area'].median(), inplace=True)
<ipython-input-77-31d0a5925258>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['usable_area'].fillna(filtered_data['usable_area'].median(), inplace=True)


In [78]:
filtered_data['number_of_parking'].fillna(0, inplace=True)

<ipython-input-78-a8cbfcadcf41>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_data['number_of_parking'].fillna(0, inplace=True)
<ipython-input-78-a8cbfcadcf41>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['number_of_parking'].fillna(0, inplace=True)


In [79]:
filtered_data.dropna(subset=['sell_price'], inplace=True)

<ipython-input-79-5b1bc757ae28>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.dropna(subset=['sell_price'], inplace=True)


In [80]:
filtered_data['sub_district'].fillna(filtered_data['sub_district'].mode()[0], inplace=True)

<ipython-input-80-a8d75c5029f4>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_data['sub_district'].fillna(filtered_data['sub_district'].mode()[0], inplace=True)
<ipython-input-80-a8d75c5029f4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['sub_district'].fillna(filtered_data['sub_district'].mode()[0], inplace=True)

In [81]:
filtered_data['zone'].fillna('unknown', inplace=True)

<ipython-input-81-ec1fe32f612d>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_data['zone'].fillna('unknown', inplace=True)
<ipython-input-81-ec1fe32f612d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['zone'].fillna('unknown', inplace=True)


In [82]:
#prompt: ตรวจสอบค่าว่างในแต่ละคอลัมน์ที่เลือก
missing_values = filtered_data.isnull().sum()
print("Missing Values:\n", missing_values)

Missing Values:
 renovate              0
number_of_rai         0
number_of_ngan        0
number_of_va          0
number_of_story       0
number_of_bedroom     0
number_of_bathroom    0
direction             0
furniture             0
usable_area           0
number_of_parking     0
sell_price            0
district              0
sub_district          0
zone                  0
dtype: int64


## **7-eleven และ school**

In [94]:
# prompt: อ่านไฟล์ .json

import json

# ระบุ path ของไฟล์
file_path_711 = '/content/drive/My Drive/Dataset/7-eleven.json'
file_path_school = '/content/drive/My Drive/Dataset/school.json'

# อ่านข้อมูลจากไฟล์ JSON
try:
    with open(file_path_711, 'r') as f:
        data_711 = json.load(f)
    with open(file_path_school, 'r') as f:
        data_school = json.load(f)
except FileNotFoundError:
    print(f"Error: File not found at {file_path_711} or {file_path_school}")
    # Handle the error appropriately, e.g., exit the program or use a default value
    exit()


# ตอนนี้ data_711 และ data_school มีข้อมูลจากไฟล์ JSON แล้ว
# คุณสามารถประมวลผลข้อมูลต่อไปได้ตามที่ต้องการ
# แก้ไข: แสดงข้อมูล 5 รายการแรกจาก data_711 โดยแปลงเป็น list ก่อน
print(list(data_711.items())[:5])  # แสดงข้อมูล 5 รายการแรกจาก data_711 เพื่อตรวจสอบ
# แก้ไข: แสดงข้อมูล 5 รายการแรกจาก data_school โดยแปลงเป็น list ก่อน
print(list(data_school.items())[:5]) # แสดงข้อมูล 5 รายการแรกจาก data_school เพื่อตรวจสอบ

[('1', []), ('4', []), ('6', [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 16.0603149, 'lng': 102.7264856}, 'viewport': {'northeast': {'lat': 16.06160307989272, 'lng': 102.7278208298927}, 'southwest': {'lat': 16.05890342010728, 'lng': 102.7251211701073}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/shopping-71.png', 'icon_background_color': '#4B96F3', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/convenience_pinlet', 'name': '7-Eleven', 'opening_hours': {'open_now': True}, 'photos': [{'height': 3072, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/100135907463246314962">nuttapong chinnapuk</a>'], 'photo_reference': 'ARywPAIuwYgKEeh9_5vpztMT_JbNv0OakveKoYeKCVvX2RyHmpDJw1w05XJIVVx9w5h9Aoai9Gb67AywD7szMyKpXsSzRol2DkngoztlEQu6goN6W2TsbB45jN-38BigXeivHTE4go3fzzTT5urf1wmSYgGzWw4g2IeDHCXM1oHEXvmTRk0Y', 'width': 1728}], 'place_id': 'ChIJzbBjEk2BGDER4YEKqD5kwlI', 'rating': 4, 'reference': 'ChIJzb

## **ประเมินราคาซื้อขาย และ คะแนนทรัพย์**
ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score') จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=1 คนเดียว (3 คะแนน)

In [100]:
# prompt: import และอ่านไฟล์ .xlsx จาก My Drive

from google.colab import drive
import pandas as pd
import json

drive.mount('/content/drive')

# Specify the path to the Excel file
appraiser_data = '/content/drive/My Drive/Dataset/คำตอบนักประเมิน(ID).xlsx'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [102]:
# prompt: import และอ่านไฟล์ .xlsx จาก My Drive

from google.colab import drive
import pandas as pd
import json

drive.mount('/content/drive')

# Specify the path to the Excel file
appraiser_data_path = '/content/drive/My Drive/Dataset/คำตอบนักประเมิน(ID).xlsx'

# Read the Excel file into a Pandas DataFrame
appraiser_data = pd.read_excel(appraiser_data_path)

# prompt: กรองข้อมูลโดยเอามาแค่คอลัมน์ ID ผู้ประเมิน ที่มีค่าเท่ากับ 1

# Now you can filter the DataFrame
filtered_appraiser_data = appraiser_data[appraiser_data['ID ผู้ประเมิน'] == 1]

# Display the filtered data
filtered_appraiser_data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2500000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2200000.0,"5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,6050000.0,5000000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
6,1,https://nayoo.co/khonkaen/posts/564,564,บ้านเดี่ยว,มั่นใจ,4400000.0,3500000.0,"5,000-18,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
8,1,https://nayoo.co/khonkaen/posts/566,566,บ้านเดี่ยว,มั่นใจ,2750000.0,2500000.0,"5,000-25,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
...,...,...,...,...,...,...,...,...,...,...
1862,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30327P,บ้านเดี่ยว,มั่นใจ,4950000.0,4500000.0,เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
1863,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30328P,บ้านเดี่ยว,มั่นใจ,3950000.0,3500000.0,เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
1864,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30330P,บ้านเดี่ยว,มั่นใจ,1700000.0,1800000.0,เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
1865,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30331P,บ้านเดี่ยว,มั่นใจ,5900000.0,5500000.0,เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี


In [103]:
# prompt: กรองข้อมูลโดยเอามาแค่คอลัมน์ ID ผู้ประเมิน ที่มีค่าเท่ากับ 1

# Filter the DataFrame to include only rows where 'ID ผู้ประเมิน' is equal to 1
filtered_appraiser_data = appraiser_data[appraiser_data['ID ผู้ประเมิน'] == 1]

# Display the filtered data
filtered_appraiser_data

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000.0,2500000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000.0,2200000.0,"5,000-15,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,6050000.0,5000000.0,"5,000-15,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
6,1,https://nayoo.co/khonkaen/posts/564,564,บ้านเดี่ยว,มั่นใจ,4400000.0,3500000.0,"5,000-18,000",NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
8,1,https://nayoo.co/khonkaen/posts/566,566,บ้านเดี่ยว,มั่นใจ,2750000.0,2500000.0,"5,000-25,000",NaN,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
...,...,...,...,...,...,...,...,...,...,...
1862,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30327P,บ้านเดี่ยว,มั่นใจ,4950000.0,4500000.0,เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
1863,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30328P,บ้านเดี่ยว,มั่นใจ,3950000.0,3500000.0,เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
1864,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30330P,บ้านเดี่ยว,มั่นใจ,1700000.0,1800000.0,เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
1865,1,https://www.baania.com/listing/%E0%B8%82%E0%B8...,30331P,บ้านเดี่ยว,มั่นใจ,5900000.0,5500000.0,เพื่อป้องกันปัญหาที่อาจเกิดขึ้นภายหลังหากต้องม...,NaN,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี


In [104]:
# prompt: เปลี่ยนชื่อคอลัมน์ 'ประเมินราคาซื้อ-ขาย' เป็น 'estimator'

# Rename the column 'ประเมินราคาซื้อ-ขาย' to 'estimator'
filtered_appraiser_data = filtered_appraiser_data.rename(columns={'ประเมินราคาซื้อ-ขาย': 'estimator'})

# Display the updated DataFrame to verify the change
print(filtered_appraiser_data.head())

   ID ผู้ประเมิน                              post id   ID property_type  \
0              1   https://nayoo.co/khonkaen/posts/25   25    บ้านเดี่ยว   
2              1   https://nayoo.co/khonkaen/posts/33   33    บ้านเดี่ยว   
4              1  https://nayoo.co/khonkaen/posts/495  495    บ้านเดี่ยว   
6              1  https://nayoo.co/khonkaen/posts/564  564    บ้านเดี่ยว   
8              1  https://nayoo.co/khonkaen/posts/566  566    บ้านเดี่ยว   

  ความมั่นใจในการเมินราคาทรัพย์  ราคาทรัพย์  estimator  \
0                        มั่นใจ   2890000.0  2500000.0   
2                        มั่นใจ   2600000.0  2200000.0   
4                        มั่นใจ   6050000.0  5000000.0   
6                        มั่นใจ   4400000.0  3500000.0   
8                        มั่นใจ   2750000.0  2500000.0   

  ปัจจัยในการประเมินราคาซื้อ-ขาย เหตุผลไม่มั่นใจราคาประเมิน  \
0                   5,000-15,000                        NaN   
2                   5,000-15,000                        NaN   
4    

In [105]:
# prompt: เปลี่ยนชื่อคอลัมน์ 'ให้คะแนนทรัพย์' เป็น 'score'

# Rename the column 'ให้คะแนนทรัพย์' to 'score'
filtered_appraiser_data = filtered_appraiser_data.rename(columns={'ให้คะแนนทรัพย์': 'score'})

# Display the updated DataFrame to verify the change
print(filtered_appraiser_data.head())

   ID ผู้ประเมิน                              post id   ID property_type  \
0              1   https://nayoo.co/khonkaen/posts/25   25    บ้านเดี่ยว   
2              1   https://nayoo.co/khonkaen/posts/33   33    บ้านเดี่ยว   
4              1  https://nayoo.co/khonkaen/posts/495  495    บ้านเดี่ยว   
6              1  https://nayoo.co/khonkaen/posts/564  564    บ้านเดี่ยว   
8              1  https://nayoo.co/khonkaen/posts/566  566    บ้านเดี่ยว   

  ความมั่นใจในการเมินราคาทรัพย์  ราคาทรัพย์  estimator  \
0                        มั่นใจ   2890000.0  2500000.0   
2                        มั่นใจ   2600000.0  2200000.0   
4                        มั่นใจ   6050000.0  5000000.0   
6                        มั่นใจ   4400000.0  3500000.0   
8                        มั่นใจ   2750000.0  2500000.0   

  ปัจจัยในการประเมินราคาซื้อ-ขาย เหตุผลไม่มั่นใจราคาประเมิน  \
0                   5,000-15,000                        NaN   
2                   5,000-15,000                        NaN   
4    

# **2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)**
zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)

บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)

ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

In [106]:
# prompt: จากคอลัมน์ 'sell_price' ให้หาค่าเฉลี่ยออกมาแบ่งตาม 'zone'

# Calculate the average 'sell_price' for each 'zone'
average_sell_price_by_zone = filtered_data.groupby('zone')['sell_price'].mean()

# Find the zone with the highest average 'sell_price'
zone_with_highest_average_price = average_sell_price_by_zone.idxmax()
highest_average_price = average_sell_price_by_zone.max()

print(f"Zone with the highest average sell price: {zone_with_highest_average_price}")
print(f"Highest average sell price: {highest_average_price}")

Zone with the highest average sell price: ในเมือง-รื่นรมย์-เทพารักษ์
Highest average sell price: 7201666.666666667


In [115]:
# prompt: # prompt: จาก filtered_appraiser_data ให้เอาออกแค่คอลัมน์ ID ผู้ประเมิน และ  post id

# Drop the specified columns
filtered_appraiser_data = filtered_appraiser_data.drop(columns=['ID ผู้ประเมิน', 'post id'])

# Display the updated DataFrame
print(filtered_appraiser_data.head())

    ID property_type ความมั่นใจในการเมินราคาทรัพย์  ราคาทรัพย์  estimator  \
0   25    บ้านเดี่ยว                        มั่นใจ   2890000.0  2500000.0   
2   33    บ้านเดี่ยว                        มั่นใจ   2600000.0  2200000.0   
4  495    บ้านเดี่ยว                        มั่นใจ   6050000.0  5000000.0   
6  564    บ้านเดี่ยว                        มั่นใจ   4400000.0  3500000.0   
8  566    บ้านเดี่ยว                        มั่นใจ   2750000.0  2500000.0   

  ปัจจัยในการประเมินราคาซื้อ-ขาย เหตุผลไม่มั่นใจราคาประเมิน  \
0                   5,000-15,000                        NaN   
2                   5,000-15,000                        NaN   
4                   5,000-15,000                        NaN   
6                   5,000-18,000                        NaN   
8                   5,000-25,000                        NaN   

                                score  
0   3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน  
2  2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี  
4   3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน 

In [118]:
# prompt: นำ filtered_data และ filtered_appraiser_data รวมกันโดยเชื่อมด้วย iD และ id

# Merge the two dataframes
merged_data = pd.merge(filtered_data, filtered_appraiser_data, left_index=True, right_index=True, how='left')

# Display the merged data
print(merged_data.head())

  renovate  number_of_rai  number_of_ngan  number_of_va  number_of_story  \
0       no              0               1          45.0              2.0   
1       no              0               0          88.0              1.0   
2       no              1               0           8.0              1.0   
3       no              0               0          61.6              2.0   
4       no              0               0          72.3              2.0   

   number_of_bedroom  number_of_bathroom direction furniture  usable_area  \
0                2.0                   2   ไม่ระบุ     ไม่มี        244.0   
1                1.0                   1   ไม่ระบุ     ไม่มี        164.0   
2               11.0                  11   ไม่ระบุ   ไม่ระบุ        489.0   
3                3.0                   3   ไม่ระบุ   ไม่ระบุ        203.0   
4                3.0                   2   ไม่ระบุ   ไม่ระบุ        164.0   

   ...  sub_district                   zone   ID property_type  \
0  ...   ศรีบุ

In [150]:
# prompt: จากคอลัมน์ score ให้แทน '1 ขายไม่ได้แน่นอน' เป็น 1, '2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี' เป็น 2, '3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน' เป็น 3, '4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน' เป็น 4, '5 ขายง่าย ใช้ระยะเวลา 1 - 3 เดือน' เป็น 5 และ NaN ให้คงค่าไว้เท่าเดิม เก็บค่าไว้ในคอลัมน์ใหม่ชื่อ score_new

# Create a mapping dictionary for score values
score = {
    '1 ขายไม่ได้แน่นอน': 1,
    '2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี': 2,
    '3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน': 3,
    '4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน': 4,
    '5 ขายง่าย ใช้ระยะเวลา 1 - 3 เดือน': 5
}

# Create the 'score_new' column using the mapping and .map()
merged_data['score_new'] = merged_data['score'].map(score)

# Display the updated DataFrame to verify the changes
print(merged_data.head())

  renovate  number_of_rai  number_of_ngan  number_of_va  number_of_story  \
0       no              0               1          45.0              2.0   
1       no              0               0          88.0              1.0   
2       no              1               0           8.0              1.0   
3       no              0               0          61.6              2.0   
4       no              0               0          72.3              2.0   

   number_of_bedroom  number_of_bathroom direction furniture  usable_area  \
0                2.0                   2   ไม่ระบุ     ไม่มี        244.0   
1                1.0                   1   ไม่ระบุ     ไม่มี        164.0   
2               11.0                  11   ไม่ระบุ   ไม่ระบุ        489.0   
3                3.0                   3   ไม่ระบุ   ไม่ระบุ        203.0   
4                3.0                   2   ไม่ระบุ   ไม่ระบุ        164.0   

   ...  estimator  ปัจจัยในการประเมินราคาซื้อ-ขาย เหตุผลไม่มั่นใจราคาประเมิน  \
